In [1]:
def read_csv(filename):
    res = {}
    with open(filename) as fhandle:
        reader = csv.DictReader(fhandle)
        for row in reader:
            res[row['filename']] = row['class']
    return res

In [2]:
# read_csv("/home/david_tyuman/my_github/cv_fall_2022/lesson_8_detection_metric/")

In [3]:
import copy
import csv
import json
import matplotlib.pyplot as plt
import os
import typing as tp
from itertools import chain

import numpy as np
import pathlib
import sklearn.metrics
import torch
from PIL import Image
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import models

from dataset import(
    ImageClassifyDataset
)
from network import ClassifyNetwork


EMBD_NUMBER_TO_SHOW = 2500

/home/david_tyuman/.pyenv/versions/3.8.2/envs/mac_env/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: /home/david_tyuman/.pyenv/versions/3.8.2/envs/mac_env/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZNK3c1010TensorImpl36is_contiguous_nondefault_policy_implENS_12MemoryFormatE
  warn(f"Failed to load image Python extension: {e}")


In [4]:
with open('./configs/learning_process/base.json') as f:
    learning_process = json.load(f)
params = learning_process["hyper_params"]["loss"]["params"]
if "weight" in params:
    learning_process["hyper_params"]["loss"]["params"]["weight"] = \
        torch.Tensor(list(params["weight"].values()))

In [5]:
# class ImageClassifyDatasetBalancedSampler(torch.utils.data.Sampler):
#     class Iterator:
#         def __init__(self, consumption: dict, quantile_class: int):
#             self.consumption = consumption
#             self.class_names = list(self.consumption)
#             self._size = 0
#             self._quantile_class = quantile_class
            
#         def __next__(self):
#             if len(self.class_names) == 0:
#                 self.class_names = list(self.consumption)
#             class_name = self.class_names.pop()
            
#             nums_of_empty_dicts = len(
#                 list(filter(lambda l: len(l) == 0, self.consumption.values()))
#             )
#             if nums_of_empty_dicts >= self._quantile_class:
#                 raise StopIteration

#             while len(self.consumption[class_name]) == 0:
#                 class_name = self.class_names.pop()
#             indx = self.consumption[class_name].pop()
#             return indx
            
#     def __init__(
#             self,
#             dataset: ImageClassifyDataset,
#             quantile_class: int
#     ):
#         self._items_groups = dict(dataset._items_groups).copy()
        
#         self._distr = {
#             class_name: len(indexes) / len(dataset)
#                 for class_name, indexes in self._items_groups.items()
#         }
#         self._quantile_class = quantile_class
        
#     def __iter__(self):
#         return self.Iterator(copy.deepcopy(self._items_groups), self._quantile_class)
    
#     def __len__(self):
#         it_ = self.Iterator(copy.deepcopy(self._items_groups), self._quantile_class)
#         len_ = 0
#         try:
#             while (i := next(it_)):
#                 len_ += 1
#         except StopIteration:
#             pass
#         return len_

In [6]:
dataset_params = learning_process["dataset_params"]
val_type = dataset_params["val_type"]
train_fraction = dataset_params["train_fraction"]
output_classes_num = learning_process["network"]["output_classes_num"]
new_size = dataset_params["new_size"]
if dataset_params["augmentation"] is None:
    transforms = None
else:
    transforms = [
        globals()[dict_["transform_type"]](**dict_["params"])
            for dict_ in dataset_params["augmentation"]
    ]

train_dataset = ImageClassifyDataset(
    mode="train",
    val_type=val_type,
    train_fraction=train_fraction,
    root_folder=pathlib.Path('/home/david_tyuman/my_github/cv_fall_2022/lesson_8_detection_metric/cropped-train'),
    classes_num=output_classes_num,
    new_size=new_size,
    transforms=transforms
)

val_dataset = ImageClassifyDataset(
    mode="val",
    val_type=val_type,
    train_fraction=train_fraction,
    root_folder=pathlib.Path('/home/david_tyuman/my_github/cv_fall_2022/lesson_8_detection_metric/cropped-train'),
    classes_num=output_classes_num,
    new_size=new_size,
    transforms=None
)

# train_dataloader = DataLoader(
#     dataset=train_dataset,
#     batch_size = dataset_params["train_batch_size"],
#     sampler=ImageClassifyDatasetBalancedSampler(train_dataset, 20),
#     drop_last=True
# )

# val_dataloader = DataLoader(
#     dataset=val_dataset,
#     batch_size = dataset_params["val_batch_size"],
#     shuffle=False
# )

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size = dataset_params["train_batch_size"],
    shuffle=True,
    drop_last=True
)

val_dataloader = DataLoader(
    dataset=val_dataset,
    batch_size = dataset_params["val_batch_size"],
    shuffle=False
)

In [7]:
len(train_dataset), len(val_dataset)

(71900, 7996)

In [8]:
len(train_dataloader), len(val_dataloader)

(70, 8)

In [9]:
network_params = learning_process["network"]
net = ClassifyNetwork(
    base_net=getattr(models, network_params["base_net"]["type"])(
        **network_params["base_net"]["params"]
    ),
    internal_features=network_params["internal_features"],
    output_classes_num=network_params["output_classes_num"],
    correct_priors=network_params["correct_priors"]
)

loss_params = learning_process["hyper_params"]["loss"]
loss = getattr(torch.nn, loss_params["loss_type"])(**loss_params["params"])

optimizer_params = learning_process["hyper_params"]["optimizer"]
optimizer = getattr(torch.optim, optimizer_params["optimizer_type"])(
    net.parameters(),
    **optimizer_params["params"]
)

epoch_nums = learning_process["hyper_params"]["epoch_nums"]

def _wrap_func(func: tp.Callable, params: dict) -> tp.Callable:
    def _wrapper(*args, **kwargs):
        kwargs.update(params)
        return func(*args, **kwargs)
    return _wrapper
metrics = []
for metric in learning_process["validation"]["metrics"]:
    metric_func = getattr(sklearn.metrics, metric["type"])
    _wraped_metric = _wrap_func(metric_func, metric["params"])
    metrics.append(
        {
            "name": metric["name"],
            "prediction_type": metric["prediction_type"],
            "func": _wraped_metric
        }
    )
main_metric = learning_process["validation"]["main_metric"]
best_main_metric_value = float("inf")

In [10]:
def get_calibration(
        y_probs: np.ndarray,
        y_true: np.ndarray
    ) -> tp.List[tp.Tuple[float, float]]:
    y_pred = y_probs.argmax(axis=1)
    y_max_probs = y_probs[range(y_true.shape[0]), y_pred]
    results = (y_true == y_pred)
    rez = [
        (i + 0.025, np.mean(results[(i < y_max_probs) & (y_max_probs < (i + 0.05))]))
            for i in np.linspace(0, 1, 20, False)
    ]
    return rez

In [ ]:
!rm -rf ./runs

# default `log_dir` is "runs".
writer = SummaryWriter('./runs')

step = 0
for epoch in range(epoch_nums):
    print(f"{epoch=}:")

    # Train model.
    if epoch > 0:
        net.train()
        loss_history = []
        for X, y in train_dataloader:
            optimizer.zero_grad()
            y_pred = net(X)
            loss_value = loss(y_pred, y)
            loss_value.backward()
            optimizer.step()
            cur_train_loss = loss_value.cpu().data.item()
            loss_history.append(cur_train_loss)
            print("train_loss: %.4f" % cur_train_loss, end='\r')
            
            backbone_grad = np.hstack(
                tuple(
                    weights.grad.numpy().reshape(-1)
                        for weights in filter(
                            lambda w: w.grad is not None, net._backbone.parameters()
                        )
                )
            ) 
            
            head_grad = np.hstack(
                tuple(
                    weights.grad.numpy().reshape(-1)
                        for weights in net._head.parameters()
                )
            )
            
            writer.add_scalars(
                "Grad_norm",
                {
                    "backbone": (backbone_grad ** 2).sum() ** 0.5,
                    "head": (head_grad ** 2).sum() ** 0.5                    
                },
                step
            )     
            
            step += 1

        train_loss = np.mean(loss_history)
        print("train_loss:\t%.5f" % train_loss)
    else:
        train_loss = -output_classes_num * np.log(1 / output_classes_num)

    # Validate model.
    net.eval()
    loss_history = []
    y_probs_history, y_true_history = [], []
    embds_history = []
    with torch.no_grad():
        for X, y in val_dataloader:
            y_logits: torch.Tensor = net(X)
            loss_value = loss(y_logits, y)
            loss_history.append(loss_value.cpu().data.item())
            
            y_probs: np.ndarray = net.predict_proba(X)
            y_true: np.ndarray = y.data.cpu().numpy().argmax(axis=1)
            embds: np.ndarray = net.backbone(X).cpu().data.numpy()               
            
            y_probs_history.extend(y_probs.tolist())
            y_true_history.extend(y_true.tolist())
            embds_history.extend(embds.tolist())
    val_loss = np.mean(loss_history)
    print("val_loss:\t%.5f" % val_loss)            
            
    # Calculate calibration and save it it TB
    probs_and_ratios = get_calibration(
        np.array(y_probs_history).copy(),
        np.array(y_true_history).copy()
    )
    
    calibration_dict = {
            "%.3f" % pair[0]: abs(pair[0] - pair[1]) for pair in probs_and_ratios
    }
    calibration_dict.update(
        {
            "E": np.mean(
                [
                    (pair[0] - pair[1]) ** 2 
                        for pair in filter(
                            lambda pair: not np.isnan(pair[1]),
                            probs_and_ratios
                        )
                ]
            ) ** 0.5
        }    
    )
    writer.add_scalars('Calibration', calibration_dict, epoch)
    
    # Calculate metrics.
    y_probs_history = np.array(y_probs_history)
    y_pred_history = y_probs_history.argmax(axis=1)
    metrics_values = {
        metric["name"]: metric["func"](y_true_history, y_pred_history)
            if metric["prediction_type"] == "categories" else
                metric["func"](y_true_history, y_probs_history)
            for metric in metrics
    }
    for name, metric_value in metrics_values.items():
        print(f"val_{name}:\t%.5f" % metric_value)
    
    # Save to TB images embeds.
    writer.add_embedding(
        np.array(embds_history[:EMBD_NUMBER_TO_SHOW]),
        metadata=y_true_history[:EMBD_NUMBER_TO_SHOW],
        global_step=epoch,
        label_img=torch.concat(
            [
                val_dataset[i][0].unsqueeze(0)
                    for i in range(len(val_dataset))
            ]
         )[:EMBD_NUMBER_TO_SHOW]
    )

    # Saving to TB.
    writer.add_scalars(
        'Loss',
        {
            "train": train_loss,
            "val": val_loss
        },
        epoch
    )

    writer.add_scalars(
        'Metric',
        metrics_values,
        epoch
    )        
    
    # Save current best model.
    if metrics_values[main_metric] < best_main_metric_value:
        best_main_metric_value = metrics_values[main_metric]
        torch.save(
            {
                'model_state_dict': net.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()
            },
            pathlib.Path("./road_signs_model.ckpt")
        )
        print("*")
    print()

writer.close()

epoch=0:
val_loss:	5.45325


/home/david_tyuman/.pyenv/versions/3.8.2/envs/mac_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/david_tyuman/.pyenv/versions/3.8.2/envs/mac_env/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


val_log_loss:	5.45618
val_pre-ion_ma:	0.00769
val_recall_ma:	0.00292
val_f1_macro:	0.00152
val_accuracy:	0.00288
*

epoch=1:
train_loss:	2.18812
val_loss:	1.27290
val_log_loss:	1.22177
val_pre-ion_ma:	0.29683
val_recall_ma:	0.20997
val_f1_macro:	0.21955
val_accuracy:	0.71486

epoch=2:
train_loss:	1.00580
val_loss:	0.92283
val_log_loss:	0.88578
val_pre-ion_ma:	0.49309
val_recall_ma:	0.33366
val_f1_macro:	0.36803
val_accuracy:	0.77426

epoch=3:
train_loss:	0.78887
val_loss:	0.80555


/home/david_tyuman/.pyenv/versions/3.8.2/envs/mac_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/david_tyuman/.pyenv/versions/3.8.2/envs/mac_env/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


val_log_loss:	0.77419
val_pre-ion_ma:	0.56010
val_recall_ma:	0.39380
val_f1_macro:	0.43360
val_accuracy:	0.79152

epoch=4:
train_loss:	0.68620
val_loss:	0.73103


...

val_log_loss:	0.63037
val_pre-ion_ma:	0.63120
val_recall_ma:	0.49638
val_f1_macro:	0.53440
val_accuracy:	0.83004

epoch=7:
train_loss:	0.53428
val_loss:	0.63087


/home/david_tyuman/.pyenv/versions/3.8.2/envs/mac_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/david_tyuman/.pyenv/versions/3.8.2/envs/mac_env/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


val_log_loss:	0.60722
val_pre-ion_ma:	0.64585
val_recall_ma:	0.51111
val_f1_macro:	0.55183
val_accuracy:	0.82904

epoch=8:
train_loss:	0.50571
val_loss:	0.62434


/home/david_tyuman/.pyenv/versions/3.8.2/envs/mac_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/david_tyuman/.pyenv/versions/3.8.2/envs/mac_env/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


val_log_loss:	0.59735
val_pre-ion_ma:	0.65856
val_recall_ma:	0.51474
val_f1_macro:	0.55500
val_accuracy:	0.83392

epoch=9:


In [14]:
ds = ImageClassifyDataset(
    mode="train",
    val_type=val_type,
    train_fraction=1.0,
    root_folder=pathlib.Path('/home/david_tyuman/my_github/cv_fall_2022/lesson_8_detection_metric/cropped-train'),
    classes_num=output_classes_num,
    new_size=new_size,
    transforms=None
)

correct_priors = {ds._class_name_to_id[k]: len(v) / len(ds) for k, v in ds._items_groups.items()}
for i in range(206):
    if i in correct_priors:
        pass
    else:
        correct_priors[i] = 0
correct_priors = dict(sorted(correct_priors.items(), key=lambda x: x[0]))
correct_priors
# p(y|x) = p(y) * p(x|y) / p(x)

{0: 0.0015019525382997897,
 1: 0,
 2: 0,
 3: 0,
 4: 0.007872734554921398,
 5: 0.005156703714829278,
 6: 0,
 7: 0.01757284469810754,
 8: 0.0017022128767397618,
 9: 0.0002878742365074597,
 10: 0,
 11: 0.0037423650745969763,
 12: 0.0006383298287774106,
 13: 0.0017397616901972564,
 14: 0.0013392410133173124,
 15: 0.001113948132572344,
 16: 0,
 17: 0,
 18: 0.011039351156503454,
 19: 0,
 20: 0,
 21: 0.007522278962651447,
 22: 0.0006508460999299089,
 23: 0,
 24: 0,
 25: 0.0011890457594873336,
 26: 0.001164013217182337,
 27: 0,
 28: 0,
 29: 0.0032041654150395515,
 30: 0.002065184740162211,
 31: 0,
 32: 0,
 33: 0.0005632322018624212,
 34: 0,
 35: 0,
 36: 0,
 37: 0.002690998297787123,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 0,
 43: 0.004280564734154401,
 44: 0,
 45: 0.015745469109842796,
 46: 0.0016896966055872634,
 47: 0.0009887854210473615,
 48: 0.011514969460298388,
 49: 0,
 50: 0.0002878742365074597,
 51: 0,
 52: 0,
 53: 0.25037548813457494,
 54: 0.00884900370481626,
 55: 0.027748573145088617,


In [18]:
net.eval()
loss_history = []
y_probs_history, y_true_history = [], []
with torch.no_grad():
    for X, y in val_dataloader:
        y_logits: torch.Tensor = net(X)
        loss_value = loss(y_logits, y)
        loss_history.append(loss_value.cpu().data.item())

        y_probs: np.ndarray = net.predict_proba(X).cpu().data.numpy()
        y_true: np.ndarray = y.data.cpu().numpy().argmax(axis=1)
        embds: np.ndarray = net.backbone(X).cpu().data.numpy()
            
        for class_id, p in list(correct_priors.items())[:-1]:
            y_probs[:, class_id] = y_probs[:, class_id] * (1/105) * p
        y_probs /= y_probs.sum(axis=1).reshape(-1, 1)

        y_probs_history.extend(y_probs.tolist())
        y_true_history.extend(y_true.tolist())
        
val_loss = np.mean(loss_history)
print("val_loss:\t%.5f" % val_loss)

y_pred_history = np.array(y_probs_history).argmax(axis=1)
metrics_values = {
    metric["name"]: metric["func"](y_true_history, y_pred_history)
        for metric in metrics[1:]
}
for name, metric_value in metrics_values.items():
    print(f"val_{name}:\t%.5f" % metric_value)

val_loss:	1.36652
val_recall_ma:	0.61171
val_f1_macro:	0.64944
val_accuracy:	0.86556


In [24]:
print("{")
for k, v in correct_priors.items():
    print(f'                "{k}": {v},')
print("}")

{
                "0": 0.0015019525382997897,
                "1": 0,
                "2": 0,
                "3": 0,
                "4": 0.007872734554921398,
                "5": 0.005156703714829278,
                "6": 0,
                "7": 0.01757284469810754,
                "8": 0.0017022128767397618,
                "9": 0.0002878742365074597,
                "10": 0,
                "11": 0.0037423650745969763,
                "12": 0.0006383298287774106,
                "13": 0.0017397616901972564,
                "14": 0.0013392410133173124,
                "15": 0.001113948132572344,
                "16": 0,
                "17": 0,
                "18": 0.011039351156503454,
                "19": 0,
                "20": 0,
                "21": 0.007522278962651447,
                "22": 0.0006508460999299089,
                "23": 0,
                "24": 0,
                "25": 0.0011890457594873336,
                "26": 0.001164013217182337,
                "27":

In [17]:
from sklearn.metrics import log_loss
log_loss?

In [10]:
net = ClassifyNetwork(
    base_net=getattr(models, network_params["base_net"]["type"])(
        **network_params["base_net"]["params"]
    ),
    internal_features=network_params["internal_features"],
    output_classes_num=output_classes_num
)
optimizer = getattr(torch.optim, optimizer_params["optimizer_type"])(
    net.parameters(),
    **optimizer_params["params"]
)

checkpoint = torch.load("./road_signs_model.ckpt")
net.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [118]:
y_probs_history = []
y_history = []
for X, y in val_dataloader:
    y_probs = net.predict_proba(X).cpu().data.numpy().tolist()
    y_probs_history.extend(y_probs)
    
    y = y.argmax(dim=1).numpy().tolist()
    y_history.extend(y)
    
y_probs = np.array(y_probs_history)
y = np.array(y_history)

In [119]:
y_pred = y_probs.argmax(axis=1)

y_max_probs = y_probs[range(y.shape[0]), y_pred]
sort_args = np.argsort(y_max_probs)

y_max_probs = y_max_probs[sort_args]
results = y[sort_args] == y_pred[sort_args]

rez = []
for i in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    mask = (i < y_max_probs) & (y_max_probs < (i + 0.1))
    bin_ = y_max_probs[mask]
    ratio = np.mean(results[mask])
    if np.any(np.isnan(ratio)):
        continue
    rez.append((i + 0.05, np.mean(results[mask])))

E = 0
for pair in rez:
    E += (pair[0] - pair[1]) ** 2
E = (E / len(rez)) ** 0.5
E

0.04203754064863384

In [120]:
rez

[(0.15000000000000002, 0.17857142857142858),
 (0.25, 0.3047619047619048),
 (0.35, 0.35398230088495575),
 (0.45, 0.43288590604026844),
 (0.55, 0.5),
 (0.65, 0.5837988826815642),
 (0.75, 0.7211538461538461),
 (0.8500000000000001, 0.7928692699490663),
 (0.9500000000000001, 0.9783167952958471)]

In [92]:
rez

[(0.05, nan),
 (0.15000000000000002, 0.0),
 (0.25, 0.42857142857142855),
 (0.35, 0.5925925925925926),
 (0.45, 0.46511627906976744),
 (0.55, 0.6129032258064516),
 (0.65, 0.6274509803921569),
 (0.75, 0.7962962962962963),
 (0.8500000000000001, 0.8289473684210527),
 (0.9500000000000001, 0.9910179640718563)]

In [86]:
i = 0.1
np.where(y_max_probs, (i < y_max_probs) & (y_max_probs < (i + 0.1)))

ValueError: either both or neither of x and y should be given

In [62]:
y_probs

array([[8.5483298e-05, 1.4772819e-14, 1.4273133e-13, ..., 2.6288084e-13,
        3.5562333e-14, 8.1335011e-14],
       [2.6610178e-06, 1.2739876e-15, 1.4398149e-14, ..., 3.6002766e-14,
        2.7983781e-15, 1.2244070e-14],
       [5.6783983e-10, 2.3162514e-14, 6.5040597e-14, ..., 5.4292894e-14,
        1.4634239e-14, 9.0630703e-14],
       ...,
       [8.6203697e-11, 2.8679354e-17, 1.0564560e-17, ..., 8.2759480e-17,
        3.0280458e-17, 4.2828560e-17],
       [9.2028388e-14, 1.3221229e-29, 1.1885842e-29, ..., 1.1067550e-28,
        4.5476420e-30, 5.3528334e-30],
       [7.8783377e-12, 5.2823892e-16, 7.0931984e-16, ..., 2.7075910e-15,
        1.8008454e-16, 1.2500440e-15]], dtype=float32)

In [35]:
(y_pred != y)

tensor([False, False, False,  ..., False, False, False])

In [21]:
i_class_probs[0]

[8.548329788027331e-05,
 2.661017788341269e-06,
 5.678398307118471e-10,
 9.541906820231816e-07,
 4.5149221250539995e-08,
 4.739329154818117e-12,
 0.009538363665342331,
 3.8132889130793046e-06,
 1.1474638085928746e-05,
 1.3246311510746575e-16,
 3.826086867775302e-06,
 9.288743385571507e-12,
 4.790371432861207e-10,
 2.2715434812000182e-14,
 1.307827375285342e-07,
 2.544249753100303e-07,
 1.9509691639996163e-09,
 0.01645510643720627,
 0.0001329982915194705,
 1.4613917714845215e-11,
 4.2789380437328883e-14,
 1.5838068547583449e-18,
 0.025467244908213615,
 3.0824320873534816e-09,
 2.0137853606883027e-09,
 1.886359677882865e-05,
 9.988932106352877e-06,
 5.618929321826727e-09,
 3.004188965736315e-13,
 6.25981328994385e-07,
 9.950562974852052e-19,
 3.0134853659546934e-05,
 1.8239314572565668e-11,
 1.4503267493637395e-06,
 6.92862032069932e-17,
 1.5633878913678956e-10,
 8.877774554787187e-16,
 5.433479865037463e-14,
 1.7962509951674477e-11,
 1.7175477706743436e-09,
 7.456380348228042e-13,
 3.58